In [ ]:
# Merge features from various datasets, including creating new ones.
# This file currently includes merging population and population density,
# income, 

In [52]:
# Merge yelp business dataset with population & population density from simple maps dataset

import pandas as pd

yelpBus_df = pd.read_csv(r'C:\Users\Garrett Pinkston\Desktop\TIM147\Project\data\RestaurantInfoWithSlope.csv')
simple_df = pd.read_csv(r'C:\Users\Garrett Pinkston\Desktop\TIM147\Project\data\uszips.csv') # Import simple maps data
income_df = pd.read_csv(r'C:\Users\Garrett Pinkston\Desktop\TIM147\Project\data\kaggle_income.csv', encoding='iso-8859-1 ',)

In [53]:
yelpBus_df['postal_code'] = pd.to_numeric(yelpBus_df['postal_code'], errors='coerce') # Convert postal code from object to float64
yelpBus_df['postal_code'] = yelpBus_df['postal_code'].astype('Int64') # Convert postal code from float64 to Int64, with capital I so that it works with nan values

In [54]:
simple_df = simple_df[['zip','population','density']]
simple_df = simple_df.rename(columns = {'zip': 'postal_code', 'density': 'pop_density'})

In [55]:
merged_df = pd.merge(yelpBus_df, simple_df, on='postal_code', how='left') # Merge 

In [56]:
income_df

,id,State_Code,State_Name,State_ab,County,City,Place,Type,Primary,Zip_Code,Area_Code,ALand,AWater,Lat,Lon,Mean,Median,Stdev,sum_w
0,1011000,1,Alabama,AL,Mobile County,Chickasaw,Chickasaw city,City,place,36611,251,10894952,909156,30.771450,-88.079697,38773,30506,33101,1638.260513
1,1011010,1,Alabama,AL,Barbour County,Louisville,Clio city,City,place,36048,334,26070325,23254,31.708516,-85.611039,37725,19528,43789,258.017685
2,1011020,1,Alabama,AL,Shelby County,Columbiana,Columbiana city,City,place,35051,205,44835274,261034,33.191452,-86.615618,54606,31930,57348,926.031000
3,1011030,1,Alabama,AL,Mobile County,Satsuma,Creola city,City,place,36572,251,36878729,2374530,30.874343,-88.009442,63919,52814,47707,378.114619
4,1011040,1,Alabama,AL,Mobile County,Dauphin Island,Dauphin Island,Town,place,36528,251,16204185,413605152,30.250913,-88.171268,77948,67225,54270,282.320328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32521,720296,72,Puerto Rico,PR,Adjuntas Municipio,Guaynabo,Adjuntas,Track,Track,970,787,589417,1691,18.397925,-66.130633,30649,13729,37977,1321.278082
32522,7202966,72,Puerto Rico,PR,Adjuntas Municipio,Aguada,Adjuntas,Track,Track,602,787,1801613,795887,18.385424,-67.203310,15520,9923,15541,238.813450
32523,7202976,72,Puerto Rico,PR,Adjuntas Municipio,Aguada,Adjuntas,Track,Track,602,787,11031227,0,18.356565,-67.180686,41933,34054,31539,313.551070
32524,7202986,72,Puerto Rico,PR,Adjuntas Municipio,Aguada,Adjuntas,Track,Track,602,787,0,33597561,18.412041,-67.213413,0,0,0,0.000000


In [57]:
# Merge income on nearest latitude & longitude

import geopandas as gpd
from geopandas.tools import sjoin_nearest # Spatial join
from shapely.geometry import Point # 

income_df = income_df[['Lat','Lon','Median']]
income_df = income_df.rename(columns = {'Median': 'median_income'})

In [58]:
left_geo = [Point(xy) for xy in zip(merged_df['latitude'],merged_df['longitude'])] # Lat & lon of restaurant
right_geo = [Point(xy) for xy in zip(income_df['Lat'],income_df['Lon'])]

In [59]:
merged_gdf = gpd.GeoDataFrame(merged_df, geometry=left_geo)
income_gdf = gpd.GeoDataFrame(income_df, geometry=right_geo)

In [60]:
sjoined_gdf = sjoin_nearest(merged_gdf, income_gdf, how='left')

In [61]:
sjoined_df = sjoined_gdf.drop(['geometry','index_right','Lat','Lon'], axis=1)

In [63]:
sjoined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19035 entries, 0 to 17601
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0.1   19035 non-null  int64  
 1   Unnamed: 0     19035 non-null  int64  
 2   business_id    19035 non-null  object 
 3   name           19035 non-null  object 
 4   address        19035 non-null  object 
 5   city           19035 non-null  object 
 6   state          19035 non-null  object 
 7   postal_code    18240 non-null  Int64  
 8   latitude       19035 non-null  float64
 9   longitude      19035 non-null  float64
 10  stars          19035 non-null  float64
 11  review_count   19035 non-null  int64  
 12  is_open        19035 non-null  int64  
 13  attributes     19035 non-null  object 
 14  categories     19035 non-null  object 
 15  hours          19035 non-null  object 
 16  is_restaurant  19035 non-null  bool   
 17  checkinDate    19035 non-null  object 
 18  starList   

In [64]:
# Create the feature for size of chain.

In [73]:
# Regularize the strings of the business names
sjoined_df['name'] = sjoined_df['name'].str.lower() # Make all lowercase
sjoined_df['name'] = sjoined_df['name'].str.strip() # Strip leading and following spaces
sjoined_df['name'] = sjoined_df['name'].str.replace('[^a-zA-Z0-9]','', regex=True)

# Calculate the number of locations for each business and create a column for it.
sjoined_df['locations'] = sjoined_df.groupby('name')['name'].transform('count')

In [74]:
# Validated that the number of locations in the dataset is actually correct.
check_location_count = sjoined_df[sjoined_df['name'] == 'arbys']
check_location_count

,Unnamed: 0.1,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,...,starList,useful,funny,cool,StarSlope,population,pop_density,median_income,locations,chain
6312,7346,7346,fbKWcur-8_W8oYVq0emvbQ,arbys,1893 W Grant Rd,Tucson,AZ,85745,32.250069,-111.007153,...,"[5, 5, 1, 1, 5, 5, 1, 5, 5, 1, 5, 5]","[0, 4, 4, 1, 0, 0, 0, 0, 0, 0, 1, 0]","[0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0]","[0, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",0.017405,37566.0,308.7,73148,8,small
8080,9398,9398,WFHYBUAdZwtAwKo6pb_ZoA,arbys,220 N Maple Ave,Marlton,NJ,8053,39.913863,-74.924932,...,"[5, 5, 2, 5, 4, 4, 4, 4, 1, 5, 4, 5]","[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]",-0.317021,46775.0,619.4,68784,8,small
8875,10319,10319,9ecu_yKxX-LfnFGcD2E4XQ,arbys,1800 Troy Rd,Edwardsville,IL,62025,38.792298,-89.950828,...,"[3, 4, 3, 1, 5, 2, 1]","[0, 1, 4, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 1, 0]","[0, 0, 2, 0, 0, 0, 0]",-0.643883,34663.0,119.2,33598,8,small
10805,12550,12550,KS2AddHgvvmz8kKb7tDpZA,arbys,402 Rising Sun Sqr Rd,Bordentown,NJ,8505,40.123421,-74.713002,...,"[1, 1, 5, 1, 5, 1, 3, 5, 4, 2, 5, 1, 1, 1, 4, ...","[0, 0, 0, 2, 0, 5, 1, 0, 1, 0, 0, 4, 0, 0, 0, ...","[1, 1, 0, 1, 0, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, ...",-0.452283,17549.0,301.3,96529,8,small
10805,12550,12550,KS2AddHgvvmz8kKb7tDpZA,arbys,402 Rising Sun Sqr Rd,Bordentown,NJ,8505,40.123421,-74.713002,...,"[1, 1, 5, 1, 5, 1, 3, 5, 4, 2, 5, 1, 1, 1, 4, ...","[0, 0, 0, 2, 0, 5, 1, 0, 1, 0, 0, 4, 0, 0, 0, ...","[1, 1, 0, 1, 0, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, ...",-0.452283,17549.0,301.3,300000,8,small
11614,13480,13480,aL-G_v3R3cYkXQXjS-Kdfg,arbys,5620 W Waters Ave,Tampa,FL,33634,28.025094,-82.539213,...,"[4, 4, 1, 4, 1, 4, 4, 4, 4, 1, 1, 1, 1, 2]","[0, 0, 0, 2, 0, 10, 0, 0, 1, 0, 5, 0, 0, 0]","[0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 1, 0, 6, 0, 0, 1, 0, 2, 0, 0, 0]",-2.349536,23662.0,1082.6,56016,8,small
11614,13480,13480,aL-G_v3R3cYkXQXjS-Kdfg,arbys,5620 W Waters Ave,Tampa,FL,33634,28.025094,-82.539213,...,"[4, 4, 1, 4, 1, 4, 4, 4, 4, 1, 1, 1, 1, 2]","[0, 0, 0, 2, 0, 10, 0, 0, 1, 0, 5, 0, 0, 0]","[0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 1, 0, 6, 0, 0, 1, 0, 2, 0, 0, 0]",-2.349536,23662.0,1082.6,52524,8,small
14116,16423,16423,92rqV6-W0Osz-kM_5968nA,arbys,10115 E Old Vail Rd,Tucson,AZ,85747,32.089490,-110.776228,...,"[4, 1, 5, 5, 2, 4, 1, 5, 4, 1, 3, 5, 5, 5, 4, ...","[1, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",0.784536,26083.0,150.3,74637,8,small


In [75]:
'''Create intervals of chain size. 1-3 locations is considered not a chain
because it requires very little infrastructure and can be run by one business
owner. 4-20 locations is considered a small chain, as ACA chain restaurant regulations
begin at 20 locations. 21-50 is a medium chain, 51-100 a large, and 101+ a mega chain.
This may be biased by the fact that not all locations are included in the dataset,
especially for large and mega chains.'''

bins = [0, 3, 20, 50, 100, float('inf')]
labels = ['not', 'small', 'medium', 'large', 'mega']
sjoined_df['chain'] = pd.cut(sjoined_df['locations'], bins=bins, labels=labels, right=False)

In [76]:
# Create and normalize the one-hots for chain.
encoded_df = pd.get_dummies(sjoined_df, columns=['chain'])
encoded_df['chain_not'] = encoded_df['chain_not'].astype(int)
encoded_df['chain_small'] = encoded_df['chain_small'].astype(int)
encoded_df['chain_medium'] = encoded_df['chain_medium'].astype(int)
encoded_df['chain_large'] = encoded_df['chain_large'].astype(int)
encoded_df['chain_mega'] = encoded_df['chain_mega'].astype(int)

In [77]:
encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19035 entries, 0 to 17601
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0.1   19035 non-null  int64  
 1   Unnamed: 0     19035 non-null  int64  
 2   business_id    19035 non-null  object 
 3   name           19035 non-null  object 
 4   address        19035 non-null  object 
 5   city           19035 non-null  object 
 6   state          19035 non-null  object 
 7   postal_code    18240 non-null  Int64  
 8   latitude       19035 non-null  float64
 9   longitude      19035 non-null  float64
 10  stars          19035 non-null  float64
 11  review_count   19035 non-null  int64  
 12  is_open        19035 non-null  int64  
 13  attributes     19035 non-null  object 
 14  categories     19035 non-null  object 
 15  hours          19035 non-null  object 
 16  is_restaurant  19035 non-null  bool   
 17  checkinDate    19035 non-null  object 
 18  starList   

In [78]:
# Normalize median income, population density, population, review count, locations
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
cols = ['review_count', 'population', 'pop_density', 'median_income', 'locations']
encoded_df[cols] = scaler.fit_transform(encoded_df[cols])

In [79]:
encoded_df.head()

,Unnamed: 0.1,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,...,StarSlope,population,pop_density,median_income,locations,chain_not,chain_small,chain_medium,chain_large,chain_mega
0,0,0,MTSW4McQd7CbVtyjqoe9mw,sthonorepastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,...,0.831331,0.146637,0.700842,0.199020,0.000000,1,0,0,0,0
1,1,1,CF33F8-E6oudUQ46HnavjQ,sonicdrivein,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,...,-3.020524,0.185734,0.003104,0.231713,0.169065,0,0,1,0,0
2,3,3,bBDDEgkFA1Otx9Lfe7BZUQ,sonicdrivein,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,...,-0.477870,0.398272,0.053533,1.000000,0.169065,0,0,1,0,0
3,5,5,il_Ro8jwPlHresjw9EGmBg,dennys,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,...,-1.412149,0.561331,0.087446,0.109217,0.100719,0,0,1,0,0
4,6,6,0bPLkL0QhhPO5kt1_EXmNQ,ziositalianmarket,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,...,-0.154502,0.275718,0.119712,0.118010,0.000000,1,0,0,0,0


In [81]:
sjoined_df.to_csv('RestaurantInfoComplete.csv') #This is unnecessary at this point; other code will be used to save.